<a href="https://colab.research.google.com/github/alexander-toschev/ai-tools/blob/main/tensorflow/tensorflow_showcase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔬 TensorFlow All-in-One Showcase
Демонстрация основных возможностей TensorFlow: от работы с тензорами до нейросетей, GPU и экспорта моделей.

In [7]:
# 📌 Установка и импорт
!pip install -q tensorflow
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.18.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## 🔹 1. Работа с тензорами
Создание и базовые операции с тензорами: сложение, матричное умножение, поэлементное произведение.

In [8]:
# Создание тензоров
a = tf.constant([[1, 2], [3, 4]])
b = tf.constant([[5, 6], [7, 8]])

# Операции
print("Matrix sum:\n", tf.add(a, b))
print("Matrix product:\n", tf.matmul(a, b))
print("Element-wise multiplication:\n", tf.multiply(a, b))

Matrix sum:
 tf.Tensor(
[[ 6  8]
 [10 12]], shape=(2, 2), dtype=int32)
Matrix product:
 tf.Tensor(
[[19 22]
 [43 50]], shape=(2, 2), dtype=int32)
Element-wise multiplication:
 tf.Tensor(
[[ 5 12]
 [21 32]], shape=(2, 2), dtype=int32)


## 🔹 2. Простая нейросеть (Keras API)
Создание, обучение и оценка модели на датасете MNIST с помощью `tf.keras`.

In [9]:
# Загружаем датасет MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

# Создание модели
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Компиляция
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Обучение
model.fit(x_train, y_train, epochs=5)

# Оценка
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8656 - loss: 0.4646
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9565 - loss: 0.1466
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9666 - loss: 0.1091
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9732 - loss: 0.0882
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9775 - loss: 0.0723
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9741 - loss: 0.0905


[0.07393353432416916, 0.9789999723434448]

## 🔹 3. Кастомный training loop (GradientTape)
Полный контроль над forward и backward проходами с использованием `tf.GradientTape`.

In [10]:
# Простая модель и ручной цикл обучения
class SimpleModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.d1 = tf.keras.layers.Dense(64, activation='relu')
        self.out = tf.keras.layers.Dense(10)

    def call(self, x):
        x = self.d1(x)
        return self.out(x)

model = SimpleModel()
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = loss_fn(y, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# Обработка данных и обучение
x_batch = tf.convert_to_tensor(x_train[:128].reshape(-1, 28*28), dtype=tf.float32)
y_batch = tf.convert_to_tensor(y_train[:128], dtype=tf.int64)

for epoch in range(5):
    loss = train_step(x_batch, y_batch)
    print(f"Epoch {epoch + 1}, Loss: {loss.numpy():.4f}")

Epoch 1, Loss: 2.4612
Epoch 2, Loss: 2.2924
Epoch 3, Loss: 2.1509
Epoch 4, Loss: 2.0278
Epoch 5, Loss: 1.9133


## 🔹 4. Использование GPU
Выполнение матричных операций на доступном GPU.

In [11]:
# Проверим работу с GPU
with tf.device('/GPU:0'):
    a = tf.random.normal([1000, 1000])
    b = tf.random.normal([1000, 1000])
    c = tf.matmul(a, b)
print("Matrix multiplication done on GPU")

Matrix multiplication done on GPU


## 🔹 5. Сохранение и загрузка модели
Сохраняем модель в формате SavedModel и загружаем обратно.

In [12]:
# Сохраняем модель
model.save('my_model')

# Загружаем модель
new_model = tf.keras.models.load_model('my_model')
new_model.summary()

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=my_model.

## 🔹 6. Экспорт модели в TensorFlow Lite
Конвертация модели в формат TFLite для мобильных и edge-устройств.

In [ ]:
# Конвертация в TFLite
converter = tf.lite.TFLiteConverter.from_saved_model('my_model')
tflite_model = converter.convert()

# Сохраняем в файл
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)
print("Model converted to TensorFlow Lite")

## 🔹 7. Style Transfer (Перенос стиля)
Пример использования TensorFlow Hub для художественного переноса стиля изображения. Мы используем предобученную модель, чтобы стилизовать обычное изображение под картину.

In [ ]:

import tensorflow_hub as hub
import PIL.Image
import requests
from io import BytesIO

def load_image(url, resize_to=(256, 256)):
    img = PIL.Image.open(requests.get(url, stream=True).raw).convert('RGB')
    img = img.resize(resize_to)
    img = np.array(img) / 255.0
    return tf.constant(img, dtype=tf.float32)[tf.newaxis, ...]

# Загружаем контент и стиль
content_image = load_image("https://tensorflow.org/example_images/YellowLabradorLooking_new.jpg")
style_image = load_image("https://tensorflow.org/example_images/Vassily_Kandinsky,_1913_-_Composition_7.jpg")

# Загружаем модель переноса стиля
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

# Применяем стиль
stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]

# Показываем результат
def show_image(image, title=''):
    plt.imshow(image[0])
    plt.axis('off')
    plt.title(title)
    plt.show()

show_image(content_image, 'Контент')
show_image(style_image, 'Стиль')
show_image(stylized_image, 'Результат')
